# Traffic Sign Classifier

The goals / steps of this project are the following:

- Load the data set (see below for links to the project data set)
- Explore, summarize and visualize the data set
- Design, train and test a model architecture
- Use the model to make predictions on new images
- Analyze the softmax probabilities of the new images
- Summarize the results with a written report

## 1. Dataset overview

Basic overview:
- Number of training examples = 34799
- Number of testing examples = 12630
- Number of validation examples = 4410
- Image data shape = (32, 32, 3)
- Number of classes = 43

Number of elements per class:

![title](./writeup/classes_distr.png)

Number of examples per class:

- count      43.000000 (number of classes)
- mean      809.279070 (average number of examples per class)
- std       626.750855
- min       180.000000 (min number of examples per class)
- 25%       285.000000
- 50%       540.000000 (median)
- 75%      1275.000000
- max      2010.000000 (max number of examples per class)

Classes example:

![title](./writeup/data_examples.png)

## 2. Data augmentation

Considering both the number of classes and confusion table, I determined the some of the classes needs to be extended by fake examples.

Unfortunately, the images in the traffic sign dataset are in a low resulotion and quality, therefore I decided not to use any techniques related to blurring, or noise adding. Otherwise even small change, could override the actual image content. I focused only on rotations and cropping images, which shouldn't decrease image quality and also increased the brightness of the images randomly.

Examples from the previous paragraph after augmentention:

![title](./writeup/augmented_data.png)

## 3. Data preprocessing

The data preprocessing consists of:
- histogram equalization (in order to improve image contrast)
- min/max normalization

I considered also converting images to grey, however it leaded to lower accuracy and difficulties in distinguishing some signs of the same shape.

## 4. Model architecture

The architecture bases on the LeNet architecture, which was extended in order to meet the accuracy of 0.93. I've also applied techniques known from AlexNet, like 'Local Response Normalization' and dropout.

There is a set of methods, I wrote in order to make the network assembling easier. 
- conv2d creates weights, biases, applies convolutions and returns the result after relu function
- fc creates fully connected layer, with a given set of inputs and outputs. Also applies relu
- lrn is a simple wrapper for tf.nn.local_response_normalization to make the code more clean

The overall architecture might be described as follows:

In [1]:
import pandas as pd
pd.read_csv("writeup/arch.csv", delimiter=';')

,layer,info
0,convolution,"filters=16,filter_size=7,strides=(1,1)"
1,max pooling,"kernel=(2,2),strides=(2,2)"
2,local response normalization,NaN
3,convolution,"filters=32,filter_size=3,strides=(1,1)"
4,convolution,"filters=32,filter_size=3,strides=(1,1)"
5,max pooling,"kernel=(2,2),strides=(2,2)"
6,local response normalization,NaN
7,flatten,NaN
8,dropout,NaN
9,fully connected,"inputs=256,outs=256"


## 5. Solution

###### Approach

The starting point of assembling the classifier, was the LeNet, that was iteratively improved till meeting given accuracy. Every modification was verified against the performance improvemenent it brings. Finally I end up with a network described above.

I prefered to avoid reusing existing architecture, that would bring me directly to the solution. Therefore it's a network, which architecture was made while experimenting.

##### Training:

Early stopping was implemented - if no improvement on validation set was made within 5 epochs, then training process is terminated. The model managed to obtain max accuracy of 0.991836734693877 on the validation set. Training was terminated after 33 epochs. 

- optimizer: Adam
- learning_rate: 0.001
- epochs: early stopping after 33 epochs.
- batch_size: 128

##### Regularization:
- dropout (training probability: 0.5, validation probability: 1.0)
- weight decay: L2 loss, ratio = 0.001

It's still to be discussed if dropout works well with weight decay, however applying such combination for the problem, showed an improvement in the increase of the final accuracy.

## 6. Accuracy

The network obtains the accuracy of 0.97553444173 on the test set

Here is the confusion matrix for the validation set:

![title](./writeup/confusion.png)


## 7. Images from Google Street View


Below you may find the extra images that were downloaded from the net:

![title](./writeup/extra_images.png)

I have tried to compare 30-speed limit and 50-speed limit as those images are quite similar. The network fails to distinguish those two particular examples, but also gives lower probability on the chosen class. The correct answer is the second one. 

However, it works well with other images and network responses gives ~100% sureness. To sum up it gives accuracy of 80% in total.

<table>
        <tr>
            <td><img src="writeup/certainty0.png"/></td>
            <td><img src="writeup/certainty1.png"/></td>
        </tr>
        <tr>
            <td><img src="writeup/certainty2.png"/></td>
            <td><img src="writeup/certainty3.png"/></td>
        </tr>
        <tr>
            <td><img src="writeup/certainty4.png"/></td>

        </tr>
</table>

The issue might be solved by collecting more examples of 50 and 30-speed limits. However I will leave, as it demonstrates interesting properties of the project.

## 8. Layers visualization

![title](./writeup/feature_map.png)